In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

##BASIC PREPROCESS RELATED LIBRARIES
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
##NLP LIBRARIES - NLTK USED IN BUILDING NLP PIPE LINE
import nltk
nltk.download('stopwords')
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()
from nltk.stem.porter import PorterStemmer
import urllib.request as url
from bs4 import BeautifulSoup as bs
import re
import requests
from sklearn.preprocessing import LabelEncoder

[nltk_data] Error loading stopwords: <urlopen error [Errno 2] No such
[nltk_data]     file or directory>


In [4]:
##TEXT CLEANING
##SCRUB WORD FUNCTION REMOVES HTML MARK UPS,SPECIAL CHARACTERS,NON ASCIII CHARACTERS,NON BREAKING SPACES ETC
def scrub_words(outtext):
    # Replace \xao characters in text -
    # \xa0 is actually non-breaking space in Latin1 (ISO 8859-1), also chr(160).  
    outtext = str(outtext)
    outtext = re.sub('\xa0', ' ', outtext)
    outtext = re.sub("(\\W|\\d)", ' ', outtext) # Replace non ascii and digits
    outtext = re.sub('\n(\w*?)[\s]', '', outtext)    # Replace new line characters and following text untill space
    outtext = re.sub("<.*?>", ' ', outtext)    # Remove html markup
    outtext = re.sub(r'\\xa0',' ',outtext)    #outtext = re.sub('\n',' ',outtext)
    outtext = re.sub(r'\\n',' ',outtext)    #outtext = re.sub(r'\xa0',' ',outtext)
    outtext = re.sub(r'_',' ',outtext)
    outtext = re.sub(r'  ',' ',outtext)
    outtext = re.sub('[^a-zA-z\s]','',outtext)
    outtext = re.sub(' +', ' ',outtext)
    " ".join(outtext.strip())
    return outtext
  
def remove_accented_chars(text):
    """Remove non-ASCII characters from list of tokenized words"""
    text = unicodedata.normalize('NFKD', text).encode(
        'ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def text_cleaning(scrapped_raw_text):   
    cleaned_text = scrub_words(scrapped_raw_text)
    return cleaned_text
  
def text_processing(text):     
    stop_words = stopwords.words('english')
    tokens = [token for token in word_tokenize(text) if not token in stop_words]
    tokens = [token for token in word_tokenize(text) if len(token)>2]
    tokens = [token.lower() for token in tokens]
    ##LEMMATIZING WORDS
    tokens = [wordnet_lemmatizer.lemmatize(token) for token in tokens]
    tokens = [token for token in word_tokenize(text) if len(token)>2]
    text = ' '.join(tokens)
    return text   

def Topic_Modeling(Test):
    #Data Collection
    df = pd.read_csv("Train_V1_yelp")
    #Test=pd.read_csv("Yelp_Test.csv")
    df["Industry"].unique(),df["Categories_1"].unique()
    df = df.loc[df["Industry"]=="Active Life"]
    df= df.loc[df["Categories_1"]!="Shopping"]
    df['Text'] = df['Text'].apply(str)
    df["label"] = df["stars"].apply(lambda x : "Negative" if 1<=x<=2 else ("Neutral" if x==3 else "Positive") )
    y = LabelEncoder().fit_transform(df['label'])

    #Remove Stop words 
    stop_words = stopwords.words('english')
    df['clean_text'] = df['Text'].apply(text_cleaning)
    df['clean_text'] = df['clean_text'].apply(text_processing)
    #Train data
    X_train=df.reset_index().copy()
    X_train.drop(columns=['Unnamed: 0','index'],inplace=True)
    X_train.reset_index(inplace=True)
    X_train.rename(columns={'index':'Id'},inplace=True)

    #Modeling
    vectorizer = TfidfVectorizer(stop_words=stop_words, max_df=0.9, max_features=1000, lowercase=False, ngram_range=(1,3))
    tfidf_vectors = vectorizer.fit_transform(X_train.clean_text)
    clf = decomposition.NMF(n_components=5, random_state=111,init=None,max_iter=10)
    W1 = clf.fit_transform(tfidf_vectors)
    H1 = clf.components_

    #Topic Extraction
    num_words=40
    vocab = np.array(vectorizer.get_feature_names())
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
    topic_words = ([top_words(t) for t in H1])
    topics = [' '.join(t) for t in topic_words]

    #Dominant topic for data
    colnames = ["Topic" + str(i) for i in range(clf.n_components)]
    docnames = [str(i) for i in range(len(X_train.clean_text))]
    df_doc_topic = pd.DataFrame(np.round(W1, 2), columns=colnames, index=docnames)
    significant_topic = np.argmax(df_doc_topic.values, axis=1)
    df_doc_topic['dominant_topic'] = significant_topic

    #Mapping dominant topic,Topics names to main data frame
    Tags=pd.read_excel("Tags.xlsx")
    Topics_Tags=pd.DataFrame.from_dict({'Id':Tags['Id'],'Topics':Tags['Topics'],'Tags':Tags['Tags']})
    doc_topic_train=pd.DataFrame.from_dict({'dominant_topic':df_doc_topic['dominant_topic']})
    Train_Tag=pd.merge(left=doc_topic_train,right=Topics_Tags,left_on='dominant_topic',right_on='Id',how='left')
    Train_Tag.reset_index(level=0, inplace=True)
    Train_Tag.rename(columns={'index':'Tag_Id'},inplace=True)
    Train_Tag=Train_Tag[['Tag_Id','Topics', 'Tags']]
    Train_Data=pd.merge(left=X_train,right=Train_Tag,left_on='Id',right_on='Tag_Id',how='left')
    Train_Data.drop(columns=['Tag_Id'],inplace=True)

    #Testing Data
    # Test modeling
    WHold = clf.transform(vectorizer.transform(Test.Text))
    #dominant topic
    colnames = ["Topic" + str(i) for i in range(clf.n_components)]
    docnames = [str(i) for i in range(len(Test.Text))]
    df_doc_topic_test = pd.DataFrame(np.round(WHold, 2), columns=colnames, index=docnames)
    significant_topic = np.argmax(df_doc_topic_test.values, axis=1)
    df_doc_topic_test['dominant_topic'] = significant_topic
    #Mapping
    doc_topic=pd.DataFrame.from_dict({'dominant_topic':df_doc_topic_test['dominant_topic']})
    Final_Tag=pd.merge(left=doc_topic,right=Topics_Tags,left_on='dominant_topic',right_on='Id',how='left')
    Final_Tag.reset_index(level=0, inplace=True)
    Final_Tag.rename(columns={'index':'Tag_Id'},inplace=True)
    Final_Tag_v1=Final_Tag[['Tag_Id','Topics', 'Tags']]
    #Final Data Frame
    Modeling_Data=pd.merge(left=Test,right=Final_Tag_v1,left_on='Id',right_on='Tag_Id',how='left')
    Modeling_Data=Modeling_Data.drop(columns=['Tag_Id'])
    return Modeling_Data



In [5]:
%%time
X_test=Topic_Modeling(pd.read_csv("fgk.csv"))

Wall time: 8.01 s


In [18]:
X_test

,Unnamed: 0,index,Id,From,To,Date,Subject,Text,Sender_Email_Type,Receiver_Email_Type,...,Label,emotion,emotions1,date_mod,Topics_x,Tags_x,Departments,emotion_updated,Topics_y,Tags_y
0,0,0,1,laxmi.saoji@gmail.com,ygayathri95@yahoo.com,2022-03-10,Golf,"Although I've never played this course, I give...",gmail,Yahoo,...,Negative,"{'Happy': 0.12, 'Angry': 0.0, 'Surprise': 0.0,...",Fear,02/17/22,tablet amazon kid buy apps kindle old work yea...,"Purchase, promotion & announcements",Procurement,"{'Happy': 0.12, 'Angry': 0.0, 'Surprise': 0.0,...",course golf play The played courses greens hol...,Golf
1,1,1,2,laxmi.saoji@gmail.com,ygayathri95@yahoo.com,2022-03-10,Golf,Easily the worse course I have played in Phoen...,gmail,Yahoo,...,Positive,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.1, ...",Sad,02/17/22,tablet amazon kid buy apps kindle old work yea...,"Purchase, promotion & announcements",Procurement,"{'Happy': 0.21, 'Angry': 0.0, 'Surprise': 0.1,...",course golf play The played courses greens hol...,Golf
2,2,2,3,laxmi.saoji@gmail.com,ygayathri95@yahoo.com,2022-03-10,Golf,Played here on a Friday afternoon. The people ...,gmail,Yahoo,...,Positive,"{'Happy': 0.06, 'Angry': 0.06, 'Surprise': 0.2...",Happy,02/14/22,tablet amazon kid buy apps kindle old work yea...,"Purchase, promotion & announcements",Procurement,"{'Happy': 0.28, 'Angry': 0.06, 'Surprise': 0.2...",course golf play The played courses greens hol...,Golf
3,3,3,4,laxmi.saoji@gmail.com,ygayathri95@yahoo.com,2022-03-10,Golf,Golf course is my favorite in the east valley....,gmail,Yahoo,...,Negative,"{'Happy': 0.09, 'Angry': 0.0, 'Surprise': 0.27...",Fear,02/19/22,deal kate symes kate symes pdx price subject c...,Feedback and reviews,Operations,"{'Happy': 0.09, 'Angry': 0.0, 'Surprise': 0.27...",course golf play The played courses greens hol...,Golf
4,4,4,5,laxmi.saoji@gmail.com,ygayathri95@yahoo.com,2022-03-10,Golf,I went out last saturday and had an good exper...,gmail,Yahoo,...,Positive,"{'Happy': 0.25, 'Angry': 0.0, 'Surprise': 0.0,...",Happy,02/13/22,tablet amazon kid buy apps kindle old work yea...,"Purchase, promotion & announcements",Procurement,"{'Happy': 0.64, 'Angry': 0.0, 'Surprise': 0.0,...",gym The equipment machines like Fitness work l...,Gyms
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,95,96,laxmi.saoji@gmail.com,ygayathri95@yahoo.com,2022-03-10,Yoga,I LOVE Metta yoga. I moved here about a year a...,gmail,Yahoo,...,Positive,"{'Happy': 0.31, 'Angry': 0.0, 'Surprise': 0.31...",Happy,02/08/22,tablet amazon kid buy apps kindle old work yea...,"Purchase, promotion & announcements",Procurement,"{'Happy': 0.31, 'Angry': 0.0, 'Surprise': 0.31...",yoga studio class classes Yoga instructors Bik...,Yoga
96,96,96,97,laxmi.saoji@gmail.com,ygayathri95@yahoo.com,2022-03-10,Yoga,"Huge fan of Bikram yoga! It's a 90 minute, tor...",gmail,Yahoo,...,Positive,"{'Happy': 0.09, 'Angry': 0.0, 'Surprise': 0.26...",Happy,02/13/22,tablet amazon kid buy apps kindle old work yea...,"Purchase, promotion & announcements",Procurement,"{'Happy': 0.4, 'Angry': 0.0, 'Surprise': 0.26,...",bike service would back shop said time get cus...,Sporting Goods
97,97,97,98,laxmi.saoji@gmail.com,ygayathri95@yahoo.com,2022-03-10,Yoga,"At the suggestion of some biker friends, I too...",gmail,Yahoo,...,Positive,"{'Happy': 0.21, 'Angry': 0.0, 'Surprise': 0.42...",Surprise,02/19/22,tablet amazon kid buy apps kindle old work yea...,"Purchase, promotion & announcements",Procurement,"{'Happy': 0.35, 'Angry': 0.0, 'Surprise': 0.42...",yoga studio class classes Yoga instructors Bik...,Yoga
98,98,98,99,laxmi.saoji@gmail.com,ygayathri95@yahoo.com,2022-03-10,Yoga,Because of my buisness I am lucky enough to tr...,gmail,Yahoo,...,Positive,"{'Happy': 0.56, 'Angry': 0.06, 'Surprise': 0.1...",Happy,02/21/22,tablet amazon kid buy apps kindle old work yea...,"Purchase, promotion & announcements",Procurement,"{'Happy': 0.56, 'Angry': 0.06, 'Surprise': 0.1...",yoga studio class c